In [1]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
import pandas as pd
import re

import os
import json

In [4]:
def add_multicolumn(df, df2, new_col_name):
    tmp = df2.copy()    # make copy, otherwise df2 will be changed !!!
    tmp.columns = pd.MultiIndex.from_product([[new_col_name], df2.columns.tolist()])
    return pd.concat([df, tmp], axis=1)

def get_valid_filename(s):
    s = str(s).strip().replace(' ', '_')
    return re.sub(r'(?u)[^-\w.]', '', s)

In [20]:
model_path = '../experiments_results/'
nets =  list(filter(lambda x: os.path.isdir(os.path.join(model_path, x)), os.listdir(model_path)))
nets = [a for a in nets if 'resnet56' in a  or 'densenet121' in a]
nets.sort()

df = pd.DataFrame()
for net in nets:
    stats_path =  os.path.join(model_path, net) +'/result_outputs'
    data_train = pd.read_csv(stats_path+'/summary.csv', delimiter=",")
    
#     if 'clean_val_acc' in data_train.columns:
#         dftmp = data_train[['clean_val_acc', 'adv_val_acc']][:200]
#     else:
#         dftmp = data_train[['val_acc']][:200]
#         dftmp.columns=['clean_val_acc']
    df = add_multicolumn(df, data_train, net)

In [62]:
df['transfer_feat_extractor_resnet56_pgd_pgd']

,clean_train_acc,adv_train_acc,clean_train_loss,adv_train_loss,train_acc,train_loss,clean_val_acc,adv_val_acc,clean_val_loss,adv_val_loss,val_acc,val_loss,curr_epoch
0,0.558650,0.204294,1.281232,2.092049,0.381472,1.686641,0.597863,0.184858,1.167150,2.183796,0.391360,1.675473,0.0
1,0.580477,0.208480,1.221088,2.082770,0.394479,1.651929,0.529940,0.192808,1.229255,2.264003,0.361374,1.746629,1.0
2,0.576718,0.204238,1.222189,2.090766,0.390478,1.656477,0.587753,0.197266,1.197861,2.221237,0.392509,1.709549,2.0
3,0.574786,0.205197,1.227219,2.098515,0.389991,1.662867,0.592302,0.193405,1.148824,2.171316,0.392854,1.660070,3.0
4,0.576515,0.205834,1.221927,2.090287,0.391175,1.656107,0.598989,0.195841,1.163820,2.173301,0.397415,1.668561,4.0
5,0.578230,0.206324,1.224329,2.087583,0.392277,1.655956,0.607881,0.198575,1.156118,2.203351,0.403228,1.679734,5.0
6,0.583011,0.207178,1.217214,2.084445,0.395095,1.650830,0.585363,0.195887,1.169194,2.173225,0.390625,1.671210,6.0
7,0.575038,0.206660,1.226728,2.094163,0.390849,1.660445,0.582721,0.203998,1.189178,2.206362,0.393359,1.697770,7.0
8,0.574254,0.207381,1.223006,2.089600,0.390818,1.656303,0.566659,0.184122,1.228377,2.227398,0.375391,1.727887,8.0
9,0.571559,0.204826,1.230797,2.098904,0.388192,1.664851,0.581893,0.191153,1.172848,2.175002,0.386523,1.673925,9.0


In [45]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [73]:
def plot(loss, layers):
    what_to_plot = [
    ('resnet56_cifar10_fgsm', loss  , '$R_{fgsm}$', {'color':'dodgerblue', 'linestyle':'-'}),
    ('transfer_%s_resnet56_nat_fgsm' % layers, loss, '$R_{nat-fgsm}$', {'color':'navy', 'linestyle':'-'}), 
    ('transfer_%s_resnet56_fgsm_fgsm' % layers, loss, '$R_{fgsm-fgsm}$', {'color':'blue', 'linestyle':'-'}), 
    ('resnet56_cifar10_pgd' , loss, '$R_{pgd}$', {'color':'brown', 'linestyle':'-'}),
    ('transfer_%s_resnet56_nat_pgd' % layers, loss, '$R_{nat-pgd}$', {'color':'pink', 'linestyle':'-'}), 
    ('transfer_%s_resnet56_pgd_pgd' % layers, loss, '$R_{pgd-pgd}$', {'color':'palevioletred', 'linestyle':'-'}), 
    ]

    title = layers+' - '+loss#'Transfer Learning Adversarial Validation Loss'

    scale=1.5
    s=100*scale
    ll=2.2
    plt.rcParams.update({'font.size': 16*scale})

    epoch=200
    fig = plt.figure(figsize=(12,8))
    for (net, acc, label, args) in what_to_plot:
        plt.plot(df[net][acc][:epoch], label=label, color=args['color'], linestyle=args['linestyle'], linewidth=3)

    plt.xlabel('Epochs',fontdict={'size':20*scale}, labelpad=20)
    plt.ylabel('Validation accuracy',fontdict={'size':20*scale}, labelpad=20)
    plt.title(title,fontdict={'size':24*scale}, pad=20)
    ax = plt.gca()
    ax.yaxis.grid(True, linestyle='--')
    ax.xaxis.grid(True, linestyle='--')
    plt.xticks([0, 50, 100])
    # plt.ylim([0,1])
    plt.legend(loc=4,fontsize=16*scale, labelspacing=0.01, borderaxespad=0.12)
    plt.tight_layout()
#     plt.savefig('images/%s_%s.pdf' %(layers, loss))

    plt.show()

In [74]:
losses = ["clean_train_acc","adv_train_acc","clean_train_loss",
          "adv_train_loss", "clean_val_acc", 
          "adv_val_acc", "clean_val_loss", "adv_val_loss", "val_acc",]
l_choices=['all_layers', '12_layers', 'feat_extractor']
interactive_plot = interactive(plot, loss=losses, layers=l_choices);
interactive_plot
# plot(loss, layers)

interactive(children=(Dropdown(description='loss', options=('clean_train_acc', 'adv_train_acc', 'clean_train_l…